In [1]:
import pandas as pd
from sklearn.cluster import DBSCAN, OPTICS
from collections import Counter
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 200)

In [2]:
comments = pd.read_csv('comments_features.csv').drop(columns=['Unnamed: 0'])
X = comments.drop(columns=['text', 'likes', 'text_cleaned'])

In [3]:
%%time
def cluster(X, method, params, label, comments):
    clusters = method(**params).fit(X)
    comments[label] = clusters.labels_
    print(Counter(clusters.labels_))
    return comments
    
n = 5000
X = X[:n]
comments = comments.loc[:n-1, ['text']]
comments_labeled = cluster(X, DBSCAN, {'eps':2.2, 'min_samples': 5}, 'DBSCAN', comments)

Counter({1: 3228, 0: 1002, -1: 740, 2: 16, 3: 6, 5: 4, 4: 4})
CPU times: total: 17.7 s
Wall time: 17.7 s


### Cluster 0 is about Kazakhstan 8-16 and covid ~4

In [4]:
comments_labeled[comments_labeled.DBSCAN == 0][:20]

,text,DBSCAN
0,"А сейчас, свежая рубрика: ""Коронановости""",0
1,А почему статистика начала рости ? Вакцина перестала помогать ?,0
2,"И всё не те мрут, кто надо.",0
6,Чё то всё резко бустанулось,0
10,Зелю предупредили. Будет как в Казахстане,0
12,"Но не сказали самое главное, уйдет ли Назарбаев",0
15,"А сейчас, свежая рубрика: ""Коронановости""",0
16,А почему статистика начала рости ? Вакцина перестала помогать ?,0
17,"И всё не те мрут, кто надо.",0
29,Чё то всё резко бустанулось,0


### Cluster 1 - COVID19
As we can see 15 out of 20 comments are about COVID19

In [5]:
comments_labeled[comments_labeled.DBSCAN == 1][:20]

,text,DBSCAN
3,"Леонида Куравлёва госпитализировали 5 января в вирусятник Проценко. Лепилы-убийцы сообщают, шо артист находится в отделении реанимации в тяжелом состоянии, которое не меняется уже несколько дней.",1
4,"Так это всё смешно, то что в России всё по сценарию..... Сейчас наверное рост долго будет до 9 мая, а на 9 мая снова победим короновирус",1
7,"Вирусолог: Гибридного варианта коронавируса «дельтакрон» не существует, он не сможет тебе навредить Гибридный вариант коронавируса «дельтакрон»:",1
8,"А это не в США все ""вакцинированы"" по самое не хочу?",1
11,Но с 1 июля цены на топливо всё равно поднимут,1
13,"Ппц, на чужих штыках удержался у власти",1
14,"Пришли, потоптали, поплевали на землю и обратно домой, вот и вся «контр-террористическая операция» :)",1
18,"Леонида Куравлёва госпитализировали 5 января в вирусятник Проценко. Лепилы-убийцы сообщают, шо артист находится в отделении реанимации в тяжелом состоянии, которое не меняется уже несколько дней.",1
19,"Так это всё смешно, то что в России всё по сценарию..... Сейчас наверное рост долго будет до 9 мая, а на 9 мая снова победим короновирус",1
20,"Умножай на три, не промахнёшься",1


In [6]:
from sklearn.cluster import KMeans
clusters = KMeans().fit(X[:n])
comments_labeled = cluster(X, KMeans, {'n_clusters': 3, 'random_state': 42}, 'Kmeans', comments_labeled)

Counter({1: 2098, 0: 1700, 2: 1202})


### Cluster 0 20/20 is about COVID 19 

In [35]:
comments_labeled[comments_labeled.Kmeans == 0][20:40]

,text,DBSCAN,Kmeans
54,На эту миссию ОДКБ длиной в 4 дня пошли наши с вами налоги. Вот как прекрасно дружить с диктаторами.,1,0
55,Лучшее за сегодняшнее выступление Токаева *ещё более важных и интересных реформ не указали в посте. Ну в принципе они важны и интересны только казахстанцам,1,0
56,"Ничего не понял, почему на страну напали террористы, а ""президент страны ввёл пятилетний мораторий на повышение зарплат министрам и депутатам"" - это всегда так работает когда на страну нападают те...",1,0
58,"Планируется ли ввод войск ОДКБ в Россию для превентивной защиты от международных террористов, которые вероломно повышают цены? Возможно также следует заранее взять под стражу пианистов, пока они в...",-1,0
62,"Ну раз зарплаты не будут расти у чиновников, то возрастёт минимальная сумма взяток, лол.",1,0
64,"Получается, что Токаев выполняет требования террористов: 1) нарушил Конституцию, сместив Назарбаева 2) снизил цены на газ 3) в отставку отправил правительство 4) другие плюшки",1,0
66,Войска пу со швабрами попросили на выход🤣😂,1,0
68,Предлагаю оставить наши танки в северном Казахстане Нам он нужнее,1,0
69,"Токаеву , ставленнику Назарбаева, теперь придётся проводить свою политику в стране, не оглядываясь на шефа...Разумеется, если он намерен остаться президентом Казахстана...",1,0
73,"Лентач- позорное уйобище. В телеге банят всех, кто имеет противоположное мнение, там одни майданутые каклы и либерасты остались)",1,0


### Cluster 1 seems also about covid19 and other

In [9]:
comments_labeled[comments_labeled.Kmeans == 1][:20]

,text,DBSCAN,Kmeans
4,"Так это всё смешно, то что в России всё по сценарию..... Сейчас наверное рост долго будет до 9 мая, а на 9 мая снова победим короновирус",1,1
8,"А это не в США все ""вакцинированы"" по самое не хочу?",1,1
9,Цирк. Уже сами в показаниях путаются.,-1,1
11,Но с 1 июля цены на топливо всё равно поднимут,1,1
13,"Ппц, на чужих штыках удержался у власти",1,1
14,"Пришли, потоптали, поплевали на землю и обратно домой, вот и вся «контр-террористическая операция» :)",1,1
19,"Так это всё смешно, то что в России всё по сценарию..... Сейчас наверное рост долго будет до 9 мая, а на 9 мая снова победим короновирус",1,1
20,"Умножай на три, не промахнёшься",1,1
21,"Москва пошла в рост. Привет, омикрон!",-1,1
22,Вам не надоела эта ересь?,1,1


### cluster 2 is about Kazakhstan 8-16 and covid ~4

In [10]:
comments_labeled[comments_labeled.Kmeans == 2][:20]

,text,DBSCAN,Kmeans
0,"А сейчас, свежая рубрика: ""Коронановости""",0,2
1,А почему статистика начала рости ? Вакцина перестала помогать ?,0,2
2,"И всё не те мрут, кто надо.",0,2
6,Чё то всё резко бустанулось,0,2
10,Зелю предупредили. Будет как в Казахстане,0,2
12,"Но не сказали самое главное, уйдет ли Назарбаев",0,2
15,"А сейчас, свежая рубрика: ""Коронановости""",0,2
16,А почему статистика начала рости ? Вакцина перестала помогать ?,0,2
17,"И всё не те мрут, кто надо.",0,2
29,Чё то всё резко бустанулось,0,2


### cluster Kmeans 2 is subclaster cluster 0 in DBSCAN

In [13]:
def cluster_difference(comments_labeled, clusters_names, clusters_numbers):
    cluster0 = comments_labeled[comments_labeled[clusters_names[0]] == clusters_numbers[0]].index
    cluster1 = comments_labeled[comments_labeled[clusters_names[1]] == clusters_numbers[1]].index
    intersection = cluster0.intersection(cluster1)
    diff0 = cluster0.difference(cluster1)
    diff1 = cluster1.difference(cluster0)
    return intersection, diff0, diff1

def cluster_similarity(func):
    def show_info(comments_labeled, clusters_names, clusters_numbers):
        intersection, diff0, diff1 = func(comments_labeled, clusters_names, clusters_numbers)
        print('intersection: {0:d}; {3:s} - {4:s} = {1:d}; {4:s} - {3:s} = {2:d}'.format(len(intersection), len(diff0), len(diff1), *clusters_names))
    return show_info  

clusters_names = ['DBSCAN', 'Kmeans']
clusters_numbers = [0, 2]
cluster_similarity(cluster_difference)(comments_labeled, clusters_names, clusters_numbers)

intersection: 1002; DBSCAN - Kmeans = 0; Kmeans - DBSCAN = 200


In [17]:
intersection, diff0, diff1 = cluster_difference(comments_labeled, clusters_names, clusters_numbers)
comments_labeled.iloc[diff1][:20]

,text,DBSCAN,Kmeans
88,По факту ничего не изменили толком 😐,-1,2
188,По факту ничего не изменили толком 😐,-1,2
252,Наконец то на лентаче годная музыка.,-1,2
269,Эх кассетник столько плёнки с ними пожевал.,-1,2
274,Горшок кала.спасибо что падох,-1,2
283,Раз два три четыре пять...,2,2
293,Раз два три четыре пять,2,2
316,Вот только что по дороге на работу слушал их песни),-1,2
317,Не панк группа а говнори 🤣☝,-1,2
325,Всё мы в детстве были панки хотя бы в самом раннем ;)!,-1,2


In [27]:
clusters_names = ['DBSCAN', 'Kmeans']
clusters_numbers = [1, 0]
cluster_similarity(cluster_difference)(comments_labeled, clusters_names, clusters_numbers)
intersection, diff0, diff1 = cluster_difference(comments_labeled, clusters_names, clusters_numbers)
comments_labeled.iloc[diff0][:20]

intersection: 1386; DBSCAN - Kmeans = 1842; Kmeans - DBSCAN = 314


,text,DBSCAN,Kmeans
4,"Так это всё смешно, то что в России всё по сценарию..... Сейчас наверное рост долго будет до 9 мая, а на 9 мая снова победим короновирус",1,1
8,"А это не в США все ""вакцинированы"" по самое не хочу?",1,1
11,Но с 1 июля цены на топливо всё равно поднимут,1,1
13,"Ппц, на чужих штыках удержался у власти",1,1
14,"Пришли, потоптали, поплевали на землю и обратно домой, вот и вся «контр-террористическая операция» :)",1,1
19,"Так это всё смешно, то что в России всё по сценарию..... Сейчас наверное рост долго будет до 9 мая, а на 9 мая снова победим короновирус",1,1
20,"Умножай на три, не промахнёшься",1,1
22,Вам не надоела эта ересь?,1,1
23,"«ЗАБУДЬ, УДАЛЁНКИ БОЛЬШЕ НЕ БУДЕТ» - говорит мне начальник",1,1
31,"А это не в США все ""вакцинированы"" по самое не хочу?",1,1


In [32]:
clusters_names = ['DBSCAN', 'Kmeans']
clusters_numbers = [0, 0]
cluster_similarity(cluster_difference)(comments_labeled, clusters_names, clusters_numbers)
intersection, diff0, diff1 = cluster_difference(comments_labeled, clusters_names, clusters_numbers)
comments_labeled.iloc[diff1][:20]

intersection: 0; DBSCAN - Kmeans = 1002; Kmeans - DBSCAN = 1700


,text,DBSCAN,Kmeans
3,"Леонида Куравлёва госпитализировали 5 января в вирусятник Проценко. Лепилы-убийцы сообщают, шо артист находится в отделении реанимации в тяжелом состоянии, которое не меняется уже несколько дней.",1,0
5,"✹ В США новый антирекорд — 1,13 миллиона новых заражений за сутки; Это ещё не предел.",-1,0
7,"Вирусолог: Гибридного варианта коронавируса «дельтакрон» не существует, он не сможет тебе навредить Гибридный вариант коронавируса «дельтакрон»:",1,0
18,"Леонида Куравлёва госпитализировали 5 января в вирусятник Проценко. Лепилы-убийцы сообщают, шо артист находится в отделении реанимации в тяжелом состоянии, которое не меняется уже несколько дней.",1,0
24,Всем уже давно наплевать адекватным!!!!!! Коронршизы успокойтесь вы не победите!!!!!!!,1,0
25,"Лол, приехали все... Щас до 6к поднимется и заговорят опять про нерабочие дни",1,0
26,"Прикол будет, когда будет подголовная 100% вакцинация и все будут всё равно дохнуть и от короны и от последствий прививки: инсульты, инфаркты, тромбы и тд. И эти такие: Мы офыблифь",-1,0
27,"Компания выпускающая прививку, за побочные эффекты - ответственности не несет Учреждение, в котором тебе ставят эту прививку - ответственности не несет Ты, не вакцинирован - безотвественный",1,0
28,"✹ В США новый антирекорд — 1,13 миллиона новых заражений за сутки; Это ещё не предел.",-1,0
30,"Вирусолог: Гибридного варианта коронавируса «дельтакрон» не существует, он не сможет тебе навредить Гибридный вариант коронавируса «дельтакрон»:",1,0
